## PROJETO PÁSCOA COM DADOS
#### Apoiei o Projeto Dado Humanizado do Prof. Cláudio Bonel. Foi uma iniciativa chamada "Páscoa com Dados", que entregou ovos de páscoa para 100 crianças do Complexo do Alemão/RJ em 2022.

In [1]:
# Importando as bibliotecas necessarias
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

In [2]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')
dtCarga

'16/04/2022 17:03'

### CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [4]:
# Manipulando o sistema de arquivos no Windows
endereco = Path('E:\\Projetos\\Python\\08 - Prof Bonel\\Python Integracao Dados com SQL\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

# Conectando no BDODS
conexaoODS = sql.connect(BDODS)

# Definição de manipulação de dados em SQL. Métdo cursor permite manipular dados no SQL
SQL_ODS = conexaoODS.cursor()

In [5]:
# Conectar no BDODS
conexaoDW = sql.connect(BDDW)

# Definição de manipulação de dados em SQL
SQL_DW = conexaoDW.cursor()

### CONSTRUINDO A TABELA DE DIMENSÃO PERIODO (dPeriodo), no DW

In [7]:
### #Definir a query de criação da tabela
qry_dPeriodo = '''
    
    CREATE TABLE IF NOT EXISTS dPeriodo (
    
    idPeriodo INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATETIME,
    mes INTEGER,
    ano INTEGER,
    trimestre INTEGER,
    semestre INTEGER

    )
'''

#executar a criação da dPeriodo
SQL_DW.execute(qry_dPeriodo)

#executar a criação do index idPeriodo
SQL_DW.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_dPeriodo_idPeriodo ON dPeriodo (idPeriodo)")

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dPeriodo ON dPeriodo (mes,ano,trimestre,semestre)")

#Confirmar a transação
conexaoDW.commit()

### DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dPeriodo

In [8]:
# Deletar os dados atuais
SQL_DW.execute("DELETE FROM dPeriodo")

# Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dPeriodo'")

# Confirmar a transação
conexaoDW.commit()

### INSERINDO DADOS NA DIMENSÃO PERÍODO (dPeriodo), NO DW

In [9]:
# Inserindo registros na tabela dPeriodo
print("Iniciando a inserção de dados na tabela dPeriodo.")
print("-------------------------------------------------")

# definindo query de carga de dados da dPeriodo
qry_dados_dPeriodo = '''

    WITH data(d) AS
        (
            VALUES('2018-01-01')
            UNION ALL
            SELECT date(d,'+1 month')
            FROM data
            WHERE d < date('now')
         )
    SELECT 
         strftime('%d/%m/%Y', d) as data,
         cast(strftime('%m', d) as integer) as mes,
         cast(strftime('%Y', d) as integer) as ano,
         case
             when cast(strftime('%m', d) as integer) in (1,2,3) then 1 
             when cast(strftime('%m', d) as integer) in (4,5,6) then 2
             when cast(strftime('%m', d) as integer) in (7,8,9) then 3
                                                                else 4
         end as trimestre,
         case
             when cast(strftime('%m', d) as integer) in (1,2,3,4,5,6) then 1
                                                                      else 2
         end as semestre                                             
    FROM data;

'''
# Carregando dados no DF dPeriodo
dPeriodo = pd.read_sql(qry_dados_dPeriodo,conexaoDW)

# Executar a carga de dados na dPeriodo
SQL_DW.executemany('''INSERT INTO dPeriodo (data,mes,ano,trimestre,semestre) VALUES (?,?,?,?,?)''',dPeriodo.values.tolist())

# confirmar a transação
conexaoDW.commit()

# Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dPeriodo), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dPeriodo.
-------------------------------------------------
Carga finalizada! 53 registros inseridos na tbLogDP!


### FECHANDO A CONEXÃO COM O DW

In [10]:
# Confirmar a transação
conexaoDW.commit()

# Fechar a conexão
conexaoDW.close()

In [11]:
!ls

01 - WebScrapingDadosIBGEdoRJ.ipynb
02 - Tabela tbLogMunic criada para manter o historico.png
03 - Tabela dMunicipio criada para manter os dados atualizados.png
04 - CargaDimensaoDelegaciasPolicia.ipynb
05 - IntegrandoDadosIBGE_DP_BPM.ipynb
06 - IntegracaoDadosIBGE_DP_BPM_PERIODO.ipynb
BPM.xlsx
DP.csv
ResponsavelDP.csv
areaBPMv1.csv
pascoaDW.db
pascoaODS.db
popDP.csv
